In [1]:
# import numpy as np
# classification
# from models import *
data = np.array([[-4, -4, 0], [-3, -3, 0], [-2, -2, 0], [-1, -1, 0], [1, 1, 0], [2, 2, 0], [3, 3, 0],
                [1, -1, 1], [2, -2, 1], [3, -3, 3], [-1, 1, 1], [-2, 2, 1], [-3, 3, 1]])

NameError: name 'np' is not defined

In [1]:
from models import *
import numpy as np
import pandas as pd

In [2]:
from dataset import *
import pandas as pd
dataset = load_dataset('taiwan_credit_risk', train_prop=0.8, valid_prop=0.1)


df = pd.DataFrame(dataset.X_train)
df['y'] = dataset.y_train

data = df.to_numpy()

Number train examples: 24000 
Number validation examples: 3000 
Number of test examples: 3000 


In [3]:
dataset2 = pd.read_csv('datasets/ames_housing/ames_housing_training.csv', index_col=0)
df2 = pd.DataFrame(dataset2)

df2=df2.select_dtypes([np.number]).dropna()
data2_train = df2.to_numpy()[:(int)(0.9*len(df2))]
data2_val = df2.to_numpy()[(int)(0.9*len(df2)):]

In [25]:
from sklearn.model_selection import StratifiedKFold, KFold
kf = KFold(n_splits=3, shuffle=True, random_state=123)


def load_data(path):
    dataset = pd.read_csv(path, index_col=0)
    df = pd.DataFrame(dataset)
#     df = df.sample(frac=1).reset_index(drop=True)

    df= df.select_dtypes([np.number]).dropna()
    data_train = df.to_numpy()[:(int)(0.9*len(df))]
    data_val = df.to_numpy()[(int)(0.9*len(df)):]
    return df, data_train, data_val

def k_fold(path):
    l1 = []
    l2 = []
    dataset = pd.read_csv(path, index_col=0)
    df = pd.DataFrame(dataset)
    df=df.select_dtypes([np.number]).dropna()
    for train_index, test_index in kf.split(df):
        b_train = df.to_numpy()[train_index]
        b_val = df.to_numpy()[test_index]
        l1.append(b_train)
        l2.append(b_val)
    return df, l1, l2
        
        

In [26]:
df_bike, b_train, b_val = k_fold('datasets/Bike-Sharing-Dataset/day.csv')

In [6]:
b_train[0]

array([[1.000e+00, 0.000e+00, 1.000e+00, ..., 3.310e+02, 6.540e+02,
        9.850e+02],
       [1.000e+00, 0.000e+00, 1.000e+00, ..., 1.310e+02, 6.700e+02,
        8.010e+02],
       [1.000e+00, 0.000e+00, 1.000e+00, ..., 1.200e+02, 1.229e+03,
        1.349e+03],
       ...,
       [1.000e+00, 1.000e+00, 1.200e+01, ..., 1.590e+02, 1.182e+03,
        1.341e+03],
       [1.000e+00, 1.000e+00, 1.200e+01, ..., 3.640e+02, 1.432e+03,
        1.796e+03],
       [1.000e+00, 1.000e+00, 1.200e+01, ..., 4.390e+02, 2.290e+03,
        2.729e+03]])

In [32]:
def check_purity_regression(data, threshold):
    return len(data) < threshold

def calculate_mse(data_below, data_above):
    # Getting the means 
    below_mean = np.mean(data_below[:,-1])
    above_mean = np.mean(data_above[:,-1]) 
    # Getting the left and right residuals 
    res_below = data_below[:,-1] - below_mean 
    res_above = data_above[:,-1] - above_mean
    # Concatenating the residuals 
    r = np.concatenate((res_below, res_above), axis=None)

    # Calculating the mse 
    n = len(r)
    r = r ** 2
    r = np.sum(r)
    mse_split = r / n
    return mse_split

def calc_mse_whole(data):
    r = data[:,-1]-np.mean(data[:,-1])
    n = len(r)
    r = r ** 2
    r = np.sum(r)
    mse = r / n
    return mse

def get_potential_splits(data, n_columns, isForest=False):
#     print(f"finding splits for {data.shape}")
    potential_splits = {}
    if isForest:
        features = list()
        while len(features) < n_features:
            index = randrange(len(data.shape[1])-1)
            if index not in features:
                features.append(index)
    else:
        features = [i for i in range(n_columns-1)]
    for column_index in features:
        potential_splits[column_index] = []
        values = data[:, column_index]
        unique_values = np.unique(values)

        for index in range(1, len(unique_values)):
            previous_value = unique_values[index - 1]
            current_value = unique_values[index]
            potential_split = (current_value + previous_value) / 2

            potential_splits[column_index].append(potential_split)

    return potential_splits


def determine_best_split(data, potential_splits):
#     print(f"Determining best split for {data.shape}")
    overall_mse = calc_mse_whole(data)
    for column_index in potential_splits:
#         print(f"column index: {column_index}")
        time_split = 0
        for value in potential_splits[column_index]:
            
            data_below, data_above = split_data(data, split_column=column_index, split_value=value)
            current_overall_mse = calculate_mse(data_below, data_above)
            if current_overall_mse < overall_mse:
                overall_mse = current_overall_mse
#                 print(f"overall mse: {overall_mse}")
                best_split_column = column_index
                best_split_value = value

    return best_split_column, best_split_value

def regression_tree(x_train, y_train, threshold):
    data = np.concatenate((x_train, y_train), axis=1)
    regression_tree_helper(data, threshold)


def regression_tree_helper(data, threshold):

#     print(data.shape)
    if len(data) <= threshold:
        return data[:, -1]
    else:
        potential_splits = get_potential_splits(data, data.shape[1])
        split_column, split_value = determine_best_split(data, potential_splits)
        data_below, data_above = split_data(data, split_column, split_value)

        # instantiate sub-tree
        question = "{} <= {}".format(split_column, split_value)
        sub_tree = {question: []}

        # find answers (recursion)
        yes_answer = regression_tree_helper(data_below, threshold)
        no_answer = regression_tree_helper(data_above, threshold)

        sub_tree[question].append(yes_answer)
        sub_tree[question].append(no_answer)

        return sub_tree


def classify_example(example, decision_tree, q):
    question = list(decision_tree.keys())[0] # decision_tree.keys() returns a dictionary
    feature_name, comparison_operator, value = question.split() # question is of form '0' <= 0.5'

    # ask question
    if example[int(feature_name)] <= float(value):
        answer = decision_tree[question][0]
    else:
        answer = decision_tree[question][1]

    # base case
    if not isinstance(answer, dict):
        return np.quantile(answer, q)

    # recursive part
    else:
        residual_tree = answer
        return classify_example(example, residual_tree, q)




In [8]:
def find_pred(main_tree, val):
    pred = []
    for x in val:
#         print(f"predict: {classify_example(x[:-1], main_tree, 0.95)}")
#         print(f"predict: {classify_example(x[:-1], main_tree, 0.05)}")
#         print(f"predict: {classify_example(x[:-1], main_tree, 0.5)}")
#         print(f"real: {x[-1]}")
        pred.append(classify_example(x[:-1], main_tree, 0.5))
    return pred
    
    

In [9]:
def calc_mse_val(pred, y):
    r = np.sum((pred-y)**2)
    n = len(y)
    mse = r / n
    return mse

In [10]:
def calc_mae_val(pred, y):
    r = np.sum(abs(pred-y))
    n = len(y)
    mae = r / n
    return mae

In [11]:
def evaluation(val, main_tree):
    accuracy = 0
    for x in val:
        top = classify_example(x[:-1], main_tree, 0.975)
        bottom = classify_example(x[:-1], main_tree, 0.025)
        if x[-1] <= top and x[-1] >= bottom:
            accuracy += 1

    percent = accuracy / len(val)
    return percent
    

In [33]:
#baseline comparison
from sklearn.tree import DecisionTreeRegressor
cmp = 0
for i in range(len(b_train)):
    main_tree = regression_tree_helper(b_train[i], 10)
    p = find_pred(main_tree, b_val[i])
    regressor = DecisionTreeRegressor(random_state=0)
    regressor.fit(b_train[i][:,:-1], b_train[i][:,-1])
    p2 = [x for x in regressor.predict(b_val[i][:,:-1])]
    qt = calc_mse_val(p, b_val[i][:,-1])
    t = calc_mse_val(p2, b_val[i][:,-1])
    print("qt = ", qt)
    print("t = ", t)
    print("accucracy = ", evaluation(b_val[i], main_tree))
    if qt < t:
        cmp += 1
print(cmp)

    
    

qt =  80217.26331967213
t =  62389.688524590165
accucracy =  0.4057377049180328
qt =  49109.33196721312
t =  52098.516393442624
accucracy =  0.5040983606557377
qt =  56272.28703703704
t =  48572.33744855967
accucracy =  0.49382716049382713
1


In [39]:
from random import seed
from random import randrange
from csv import reader
from math import sqrt
from functools import reduce
 


# Evaluate an algorithm using a cross validation split
def evaluate_algorithm(path, algorithm, *args):
    df_bike, b_train, b_val = k_fold(path)
    scores = list()
    for i in range(len(b_train)):

        predicted = algorithm(b_train[i], b_val[i], *args)
        actual = b_val[i][:,-1]
        mae = calc_mae_val(predicted, actual)
        mse = calc_mee_val(predicted, actual)
        print("mae = ", mae)
        print("mse = ", mse)
        scores.append((mae, mse))
    return scores

# Split a dataset based on an attribute and an attribute value
# def test_split(index, value, dataset):
#     left, right = list(), list()
#     for row in dataset:
#         if row[index] < value:
#             left.append(row)
#         else:
#             right.append(row)
#     return left, right

# Calculate the Gini index for a split dataset
# def gini_index(groups, classes):
#     # count all samples at split point
#     n_instances = float(sum([len(group) for group in groups]))
#     # sum weighted Gini index for each group
#     gini = 0.0
#     for group in groups:
#         size = float(len(group))
#         # avoid divide by zero
#         if size == 0:
#             continue
#         score = 0.0
#         # score the group based on the score for each class
#         for class_val in classes:
#             p = [row[-1] for row in group].count(class_val) / size
#             score += p * p
#         # weight the group score by its relative size
#         gini += (1.0 - score) * (size / n_instances)
#     return gini

# Select the best split point for a dataset
# def get_split(dataset, n_features):
#     class_values = list(set(row[-1] for row in dataset))
#     b_index, b_value, b_score, b_groups = 999, 999, 999, None
#     features = list()
#     while len(features) < n_features:
#         index = randrange(len(dataset[0])-1)
#         if index not in features:
#             features.append(index)
#     for index in features:
#         for row in dataset:
#             groups = test_split(index, row[index], dataset)
#             gini = gini_index(groups, class_values)
#             if gini < b_score:
#                 b_index, b_value, b_score, b_groups = index, row[index], gini, groups
#     return {'index':b_index, 'value':b_value, 'groups':b_groups}

# Create a terminal node value
# def to_terminal(group):
#     outcomes = [row[-1] for row in group]
#     return max(set(outcomes), key=outcomes.count)

# # Create child splits for a node or make terminal
# def split(node, max_depth, min_size, n_features, depth):
#     left, right = node['groups']
#     del(node['groups'])
#     # check for a no split
#     if not left or not right:
#         node['left'] = node['right'] = to_terminal(left + right)
#         return
#     # check for max depth
#     if depth >= max_depth:
#         node['left'], node['right'] = to_terminal(left), to_terminal(right)
#         return
#     # process left child
#     if len(left) <= min_size:
#         node['left'] = to_terminal(left)
#     else:
#         node['left'] = get_split(left, n_features)
#         split(node['left'], max_depth, min_size, n_features, depth+1)
#     # process right child
#     if len(right) <= min_size:
#         node['right'] = to_terminal(right)
#     else:
#         node['right'] = get_split(right, n_features)
#         split(node['right'], max_depth, min_size, n_features, depth+1)

# Build a decision tree
# def build_tree(train, max_depth, min_size, n_features):
#     root = get_split(train, n_features)
#     split(root, max_depth, min_size, n_features, 1)
#     return root

# Make a prediction with a decision tree
# def predict(node, row):
#     if row[node['index']] < node['value']:
#         if isinstance(node['left'], dict):
#             return predict(node['left'], row)
#         else:
#             return node['left']
#     else:
#         if isinstance(node['right'], dict):
#             return predict(node['right'], row)
#         else:
#             return node['right']

# Create a random subsample from the dataset with replacement
def subsample(dataset, ratio):
    sample = list()
    n_sample = round(len(dataset) * ratio)
    while len(sample) < n_sample:
        index = randrange(len(dataset))
        sample.append(dataset[index])
    return sample

# Make a prediction with a list of bagged trees
# return: m x n matrix, m = number of trees, n = number of samples
def bagging_predict(trees, val):
    predictions = [find_pred(tree, val) for tree in trees]
    ##todo: return mean of the column
    result = []
    for i in range(len(predictions[0])):
        retult.append(reduce(lambda x, y : x[i] + y[i], predictions))
    
    return result

# Random Forest Algorithm
def random_forest(train, test, threshold, sample_size, n_trees, n_features):
    trees = list()
    for i in range(n_trees):
        sample = subsample(train, sample_size)
        tree = regression_tree_helper(sample, n_features, threshold)
        trees.append(tree)

    return bagging_predict(trees, test)

# main_tree = regression_tree_helper(b_train[i], 10)
#     p = find_pred(main_tree, b_val[i])
#     regressor = DecisionTreeRegressor(random_state=0)
#     regressor.fit(b_train[i][:,:-1], b_train[i][:,-1])
#     p2 = [x for x in regressor.predict(b_val[i][:,:-1])]
#     qt = calc_mse_val(p, b_val[i][:,-1])
#     t = calc_mse_val(p2, b_val[i][:,-1])

# Test the random forest algorithm
seed(2)
# load and prepare data
# filename = 'sonar.all-data.csv'
# dataset = load_csv(filename)
# convert string attributes to integers
# for i in range(0, len(dataset[0])-1):
# str_column_to_float(dataset, i)
# # convert class column to integers
# str_column_to_int(dataset, len(dataset[0])-1)
# evaluate algorithm


In [41]:
path = 'datasets/Bike-Sharing-Dataset/day.csv'
threshold = 1
sample_size = 1.0
n_features = int(sqrt(df.shape[1]-1))
for n_trees in [10, 100, 1000]:
    scores = evaluate_algorithm(path, random_forest, threshold, sample_size, n_trees, n_features)
    print('Trees: %d' % n_trees)
    print('Scores: %s' % scores)
    print('Mean Accuracy: %.3f%%' % (sum(scores)/float(len(scores))))

TypeError: regression_tree_helper() takes 2 positional arguments but 3 were given